#Setup

In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.

In [12]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam import PTransform, DoFn
from typing import Dict, List

In [3]:
!mkdir -p data

In [4]:
from google.colab import files
uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [6]:
import pandas as pd

columns = ['EmployeeID', 'Name', 'DepartmentID', 'Department', 'StartDate']

df = pd.read_csv('dept_data.txt', header=None, names=columns, delimiter=',')
df.head()

,EmployeeID,Name,DepartmentID,Department,StartDate
0,149633CM,Marco,10,Accounts,1-01-2019
1,212539MU,Rebekah,10,Accounts,1-01-2019
2,231555ZZ,Itoe,10,Accounts,1-01-2019
3,503996WI,Edouard,10,Accounts,1-01-2019
4,704275DC,Kyle,10,Accounts,1-01-2019


#Code

`beam.PTransform` to klasa bazowa w Apache Beam, która pozwala na zdefiniowanie własnych transformacji danych w potoku przetwarzania. Transformacja ta przyjmuje dane wejściowe (np. PCollection) i przekształca je, generując nowe dane (np. w postaci PCollection).

In [10]:
class MyTransform(beam.PTransform):
  def expand(self, input_coll):

    a = (input_coll
                    | 'Group and sum1' >> beam.CombinePerKey(sum)
                    | 'count filter accounts' >> beam.Filter(filter_on_count)
                    | 'Regular accounts employee' >> beam.Map(format_output)
              )
    return a

def SplitRow(element):
    return element.split(',')

def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

def format_output(element):
  name, count = element
  return ', '.join((name,str(count),'Regular employee'))

p = beam.Pipeline()

input_collection = (
                      p
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'composite accounts' >> MyTransform()
                      | 'Write results for account' >> beam.io.WriteToText('data/Account')
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                | 'composite HR' >> MyTransform()
                | 'Write results for hr' >> beam.io.WriteToText('data/HR')
           )
p.run()

In [8]:
!{('head -n 20 data/Account-00000-of-00001')}

Accounts, Marco, 31, Regular employee
Accounts, Rebekah, 31, Regular employee
Accounts, Itoe, 31, Regular employee
Accounts, Edouard, 31, Regular employee
Accounts, Kyle, 62, Regular employee
Accounts, Kumiko, 31, Regular employee
Accounts, Gaston, 31, Regular employee


In [9]:
!{('head -n 20 data/HR-00000-of-00001')}

HR, Beryl, 62, Regular employee
HR, Olga, 31, Regular employee
HR, Leslie, 31, Regular employee
HR, Mindy, 31, Regular employee
HR, Vicky, 31, Regular employee
HR, Richard, 31, Regular employee
HR, Kirk, 31, Regular employee
HR, Kaori, 31, Regular employee
HR, Oscar, 31, Regular employee


In [16]:
class PrzetwarzanieDanych(beam.PTransform):
    """Własna transformacja łącząca kilka operacji na danych"""
    def __init__(self, min_wartosc: float = 0.0):
        super().__init__()
        self.min_wartosc = min_wartosc

    def expand(self, wejscie):
        return (
            wejscie
            | "Parsowanie" >> beam.Map(self.parsuj_dane)
            | "Filtrowanie" >> beam.Filter(lambda x: x['wartosc'] > self.min_wartosc)
            | "Wzbogacenie" >> beam.Map(self.dodaj_metadane)
        )

    @staticmethod
    def parsuj_dane(rekord: str) -> Dict:
        # Przykład parsowania danych wejściowych
        id, wartosc = rekord.split(',')
        return {
            'id': id,
            'wartosc': float(wartosc)
        }

    @staticmethod
    def dodaj_metadane(rekord: Dict) -> Dict:
        rekord['status'] = 'przetworzony'
        return rekord

class AnalizaGrupowa(PTransform):
    """Transformacja do analizy grup danych"""
    def expand(self, wejscie):
        return (
            wejscie
            | "Grupowanie" >> beam.GroupByKey()
            | "Agregacja" >> beam.Map(self.oblicz_statystyki)
        )

    @staticmethod
    def oblicz_statystyki(grupa_danych):
        klucz, wartosci = grupa_danych
        wartosci_lista = list(wartosci)
        return {
            'klucz': klucz,
            'suma': sum(wartosci_lista),
            'srednia': sum(wartosci_lista) / len(wartosci_lista),
            'liczba': len(wartosci_lista)
        }

# Przykład użycia własnych transformacji
with beam.Pipeline() as pipeline:

    dane_wejsciowe = [
        "001,10.5",
        "002,15.3",
        "003,8.7",
        "004,12.1"
    ]

    wyniki = (
        pipeline
        | "Utworzenie danych" >> beam.Create(dane_wejsciowe)
        | "Przetwarzanie" >> PrzetwarzanieDanych(min_wartosc=10.0)
        | "Konwersja do par" >> beam.Map(lambda x: (x['status'], x['wartosc']))
        | "Analiza grup" >> AnalizaGrupowa()
        | "Zapis wyników" >> beam.io.WriteToText('data/wyniki.txt')
    )

In [17]:
!{('head -n 20 /content/data/wyniki.txt-00000-of-00001')}

{'klucz': 'przetworzony', 'suma': 37.9, 'srednia': 12.633333333333333, 'liczba': 3}


In [ ]:
# Definicja klasy `PrzetwarzanieDanych`, która dziedziczy po `beam.PTransform`.
# Ta klasa pozwala na zdefiniowanie niestandardowej sekwencji operacji przetwarzania danych.
class PrzetwarzanieDanych(beam.PTransform):
    """Własna transformacja łącząca kilka operacji na danych"""

    def __init__(self, min_wartosc: float = 0.0):
        """
        Konstruktor inicjalizujący klasę.
        :param min_wartosc: Minimalna wartość, która będzie używana do filtrowania rekordów.
        """
        super().__init__()  # Wywołanie konstruktora nadrzędnej klasy `PTransform`.
        self.min_wartosc = min_wartosc  # Przechowujemy minimalną wartość do wykorzystania w transformacji.

    def expand(self, wejscie):
        """
        Metoda `expand` definiuje logikę transformacji.
        :param wejscie: Dane wejściowe (kolekcja PCollection).
        :return: Przetworzone dane wyjściowe (PCollection).
        """
        return (
            wejscie
            # Parsowanie danych wejściowych na słowniki.
            | "Parsowanie" >> beam.Map(self.parsuj_dane)

            # Filtrowanie danych na podstawie pola 'wartosc', aby zostawić tylko rekordy większe niż `min_wartosc`.
            | "Filtrowanie" >> beam.Filter(lambda x: x['wartosc'] > self.min_wartosc)

            # Dodanie dodatkowych metadanych, np. oznaczenie rekordów jako przetworzone.
            | "Wzbogacenie" >> beam.Map(self.dodaj_metadane)
        )

    @staticmethod
    def parsuj_dane(rekord: str) -> Dict:
        """
        Funkcja pomocnicza do parsowania danych wejściowych.
        :param rekord: Wiersz danych w formacie tekstowym (np. "001,10.5").
        :return: Słownik z id i wartoscią (np. {'id': '001', 'wartosc': 10.5}).
        """
        id, wartosc = rekord.split(',')  # Rozdzielanie tekstu na dwa elementy za pomocą przecinka.
        return {
            'id': id,
            'wartosc': float(wartosc)  # Konwersja wartości na liczbę zmiennoprzecinkową.
        }

    @staticmethod
    def dodaj_metadane(rekord: Dict) -> Dict:
        """
        Funkcja pomocnicza do wzbogacania danych.
        Dodaje pole 'status', aby oznaczyć rekord jako przetworzony.
        :param rekord: Słownik reprezentujący rekord.
        :return: Zaktualizowany słownik z dodatkowym polem 'status'.
        """
        rekord['status'] = 'przetworzony'  # Dodanie pola 'status'.
        return rekord

# Definicja klasy `AnalizaGrupowa`, która pozwala na grupowanie i analizę danych.
class AnalizaGrupowa(PTransform):
    """Transformacja do analizy grup danych"""

    def expand(self, wejscie):
        """
        Metoda `expand` definiuje kroki przetwarzania grup danych.
        :param wejscie: Dane wejściowe w formie PCollection par klucz-wartość.
        :return: Wyniki analizy grupowej jako PCollection.
        """
        return (
            wejscie
            # Grupowanie elementów według klucza.
            | "Grupowanie" >> beam.GroupByKey()

            # Obliczanie statystyk dla każdej grupy.
            | "Agregacja" >> beam.Map(self.oblicz_statystyki)
        )

    @staticmethod
    def oblicz_statystyki(grupa_danych):
        """
        Funkcja obliczająca statystyki dla każdej grupy.
        :param grupa_danych: Para (klucz, lista wartości).
        :return: Słownik zawierający klucz oraz statystyki (suma, średnia, liczba elementów).
        """
        klucz, wartosci = grupa_danych  # Rozpakowanie klucza i wartości grupy.
        wartosci_lista = list(wartosci)  # Konwersja wartości na listę.
        return {
            'klucz': klucz,  # Zachowanie klucza grupy.
            'suma': sum(wartosci_lista),  # Obliczenie sumy wartości w grupie.
            'srednia': sum(wartosci_lista) / len(wartosci_lista),  # Obliczenie średniej wartości.
            'liczba': len(wartosci_lista)  # Obliczenie liczby elementów w grupie.
        }

# Przykład użycia potoku Apache Beam z wykorzystaniem własnych transformacji.
with beam.Pipeline() as pipeline:

    # Dane wejściowe w formacie tekstowym.
    dane_wejsciowe = [
        "001,10.5",  # Przykładowy rekord: id=001, wartosc=10.5
        "002,15.3",  # Przykładowy rekord: id=002, wartosc=15.3
        "003,8.7",   # Przykładowy rekord: id=003, wartosc=8.7
        "004,12.1"   # Przykładowy rekord: id=004, wartosc=12.1
    ]

    wyniki = (
        pipeline
        # Tworzenie PCollection z danych wejściowych.
        | "Utworzenie danych" >> beam.Create(dane_wejsciowe)

        # Zastosowanie niestandardowej transformacji do przetwarzania danych.
        | "Przetwarzanie" >> PrzetwarzanieDanych(min_wartosc=10.0)

        # Konwersja danych na pary klucz-wartość, aby przygotować dane do grupowania.
        | "Konwersja do par" >> beam.Map(lambda x: (x['status'], x['wartosc']))

        # Zastosowanie niestandardowej transformacji do grupowania i analizy danych.
        | "Analiza grup" >> AnalizaGrupowa()

        # Zapisanie wyników do pliku tekstowego.
        | "Zapis wyników" >> beam.io.WriteToText('data/wyniki.txt')
    )